In [1]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import linecache
from google.cloud import storage
import subprocess
import re

# RAW EXAMPLE

Example taken from the simplfied database on : 
https://console.cloud.google.com/storage/browserquickdraw_dataset/full/raw

In [ ]:
#RAW DATA: Example 
df = pd.DataFrame()
with open(data_path, 'r') as f:
     for i, line in enumerate(f):
            if i > 0:
                break

            json_line = json.loads(line)
            df = pd.concat([df, pd.DataFrame(json_line)], ignore_index=True)
#             print(type(json_line))
#             print(json_line.keys())
#             print(json_line['word'], len(json_line['drawing']))
            coords = json_line['drawing']
    
            for vector in coords:
                print(len(vector))
                print(vector)
           

In [ ]:
df

In [ ]:
df["drawing"][0][0:1]

In [ ]:
drawing_test= df["drawing"][0][2]
drawing_test

In [ ]:

# Extract data from the first row
first_row = df.loc[0]  # Change the index if needed

# Extract x, y, and time lists
x_values = first_row['drawing'][0]
y_values = first_row['drawing'][1]

# Create a scatter plot of the points
plt.scatter(x_values, y_values, color='blue', marker='o', label='Points')

# Connect the points with lines
for i in range(len(x_values) - 1):
    plt.plot([x_values[i], x_values[i + 1]], [y_values[i], y_values[i + 1]], color='red')

# Customize the plot (add labels, title, etc.)
plt.xlabel('X-coordinate')
plt.ylabel('Y-coordinate')
plt.title('Drawing from First RAW Row')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()


In [ ]:
# Extract data from the first drawing 



# Create a new figure for the combined drawing
plt.figure(figsize=(8, 6))

# Iterate through each row and plot the points
num_rows = df.shape[0]
for i in range(num_rows):
    x_values = df.loc[i, 'drawing'][0]
    y_values = df.loc[i, 'drawing'][1]
    plt.plot(x_values, y_values, label=f"Row {i}")

    # Create a scatter plot of the points
    plt.scatter(x_values, y_values, color='blue', marker='o', label='Points')

# Connect the points with lines
for i in range(len(x_values) - 1):
    plt.plot([x_values[i], x_values[i + 1]], [y_values[i], y_values[i + 1]], color='red')

# Customize the plot (add labels, title, etc.)
plt.xlabel('X-coordinate')
plt.ylabel('Y-coordinate')
plt.title('Drawing from First RAW drawing')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()

# SIMPLIFIED EXAMPLE

Example taken from the simplfied database on :
https://console.cloud.google.com/storage/browser/quickdraw_dataset/full/simplified

In [ ]:
#command to download the examle in terminal: gsutil -m cp 'gs://quickdraw_dataset/full/simplified/axe.ndjson' .

In [ ]:
# # Download this face file (simplified) and rename it full_simplified_face:
# gsutil -m cp 'gs://quickdraw_dataset/full/simplified/face.ndjson' full_simplified_face.ndjson


In [2]:
path_data = '/Users/loicregne/code/rs-uk/raw_data'
file_name = 'full_simplified_face.ndjson'

# Processing Simplified file

In [33]:
##Define the function to process the Simplified file and return an array and class name (the drawing to guess)
def parse_line(json_drawing: json) -> np.array:
    """Parse an ndjson line and return ink (as np array) and classname."""
    inkarray = json_drawing["drawing"]
    stroke_lengths = [len(stroke[0]) for stroke in inkarray]
    total_points = sum(stroke_lengths)
    np_ink = np.zeros((total_points, 3), dtype=np.float32)
    current_t = 0
    for stroke in inkarray:
        for i in [0, 1]:
            np_ink[current_t:(current_t + len(stroke[0])), i] = stroke[i]
        current_t += len(stroke[0])
        np_ink[current_t - 1, 2] = 1  # stroke_end

    # Preprocessing.
    # 1. Size normalization.
    lower = np.min(np_ink[:, 0:2], axis=0)
    upper = np.max(np_ink[:, 0:2], axis=0)
    scale = upper - lower
    scale[scale == 0] = 1
    np_ink[:, 0:2] = (np_ink[:, 0:2] - lower) / scale

    # 2. Compute deltas.
    np_ink[1:, 0:2] -= np_ink[0:-1, 0:2]
    np_ink = np_ink[1:, :]
    return np.round(np_ink,decimals=4)

In [48]:
##Define the function to process the Simplified file and return an array and class name (the drawing to guess)
def get_drawing_data(json_drawing: json) -> np.array:
    '''
    Extracts the drawing data (strokes list) from a drawing JSON file.
    Transforms the strokes from coordinates to deltas.
    Returns an np.array of deltas (d_x, d_y, end_of_stroke)
    '''
    # --- Data extraction ---
    list_strokes = json_drawing['drawing']
    
    x = []
    y = []
    stroke_delimiter = []
    list_points = [x, y, stroke_delimiter]
    
    for stroke in list_strokes:
        # Creating the third list to pass to the model with 0 all along and a 1 at the end of the stroke
        stroke_delimiter = [0.] * len(stroke[0])
        stroke_delimiter[-1] = 1
        # Appending x, y, and the delimiter to the new list of points
        list_points[0] += stroke[0]
        list_points[1] += stroke[1]
        list_points[2] += stroke_delimiter

    np_points = np.asarray(list_points)
    np_points = np_points.T

    # --- Preprocessing ---
    # 1. Size normalization
    lower = np.min(np_points[:, 0:2], axis=0) # returns (x_min, y_min)
    upper = np.max(np_points[:, 0:2], axis=0) # returns (x_max, y_max)
    scale = upper - lower # returns (width, heigth)
    scale[scale == 0] = 1
    np_points[:, 0:2] = (np_points[:, 0:2] - lower) / scale

    # 2. Compute deltas
    np_points[1:, 0:2] -= np_points[0:-1, 0:2]
    np_points = np_points[1:, :]
    return np.round(np_points,decimals=4)

# Define the function to process a  CLASS (one of the word to guess) full file 

In [5]:
ndjson_filepath = path_data + '/' + file_name

In [6]:
def drawings_extraction(ndjson_filepath, nb_drawings_to_load):
    list_drawings = []  # Initialize an empty list
    with open(ndjson_filepath, 'r') as f:
        for i, line in enumerate(f):
            json_line = json.loads(line)
            np_ink = parse_line(json_line)
            length_np_ink = len(np_ink)
            dict_drawing_info = {'key_id': json_line['key_id'],
                                 'class': json_line['word'],
                                 'length_np_ink': length_np_ink,
                                 'np_ink': np_ink.tolist()  #need to be transformed to list to dump as Json file later
                                }
            list_drawings.append(dict_drawing_info)
            i += 1
            if i == nb_drawings_to_load:
                break
    return list_drawings

In [71]:
# Trying a more efficient way to load the data

def drawings_extraction_v2(ndjson_filepath: object, nb_drawings_to_load: str) -> list:
    '''
    Extract drawing(s) information from a list of JSON drawings (as NDJSON),
    as a list of dictionaries. We specify the number of drawings to load (in
    order of the NDJSON) as a string (number or 'all'). Each dictionary contains:
        - key_id, as string
        - class, as string
        - length, as integer
        - list_deltas, as list
    '''
    list_drawings = []  # Initialize the list to return

    if nb_drawings_to_load == 'all':
        # Getting the number of line in the file using a shell command (fastest way)
        nb_drawings_to_load = int(re.search(r'\d+', str(subprocess.check_output(['wc', '-l', ndjson_filepath]))).group())
    elif (isinstance(nb_drawings_to_load, str) and nb_drawings_to_load.isnumeric()) or isinstance(nb_drawings_to_load, int):
        # We also escape a number of drawings entered as an integer instead of a string...
        nb_drawings_to_load = int(nb_drawings_to_load)
    else:
        nb_drawings_to_load = 0
        
    for i in range(0, int(nb_drawings_to_load)):
        json_drawing = json.loads(linecache.getline(ndjson_filepath, i+1 , module_globals=None))
        np_deltas = get_drawing_data(json_drawing)
        dict_drawing = {'key_id': json_drawing['key_id'],
                        'class': json_drawing['word'],
                        'length': len(np_deltas),
                        'list_deltas': np_deltas.tolist()  # need to be transformed to list to dump as Json file later
                       }
        list_drawings.append(dict_drawing)
    linecache.clearcache()
    
    return list_drawings

In [74]:
%%time
list_drawings = drawings_extraction(ndjson_filepath, 50000)

CPU times: user 2.89 s, sys: 148 ms, total: 3.04 s
Wall time: 3.08 s


In [75]:
%%time
list_drawings = drawings_extraction_v2(ndjson_filepath, 50000)

CPU times: user 1.82 s, sys: 81.8 ms, total: 1.91 s
Wall time: 1.91 s


In [76]:
list_drawings[0]

{'key_id': '5152530285723648',
 'class': 'face',
 'length': 65,
 'list_deltas': [[-0.0591, 0.018, 0.0],
  [-0.063, 0.0676, 0.0],
  [-0.0433, 0.0991, 0.0],
  [-0.0354, 0.1261, 0.0],
  [-0.0315, 0.1757, 0.0],
  [0.0, 0.1351, 0.0],
  [0.0512, 0.1351, 0.0],
  [0.0748, 0.0946, 0.0],
  [0.0669, 0.0495, 0.0],
  [0.1417, 0.0676, 0.0],
  [0.0984, 0.0225, 0.0],
  [0.1024, 0.0045, 0.0],
  [0.122, 0.0, 0.0],
  [0.0866, -0.0225, 0.0],
  [0.0787, -0.036, 0.0],
  [0.0787, -0.0631, 0.0],
  [0.0984, -0.2297, 0.0],
  [0.0, -0.1757, 0.0],
  [-0.063, -0.1757, 0.0],
  [-0.063, -0.0811, 0.0],
  [-0.063, -0.0495, 0.0],
  [-0.0984, -0.0586, 0.0],
  [-0.1732, -0.0721, 0.0],
  [-0.2008, -0.036, 0.0],
  [-0.1417, 0.0, 0.0],
  [-0.0472, 0.0315, 1.0],
  [0.1496, 0.2432, 0.0],
  [0.0, 0.027, 0.0],
  [-0.0197, 0.018, 0.0],
  [-0.0472, -0.0045, 0.0],
  [-0.0118, -0.027, 0.0],
  [0.0, -0.0405, 0.0],
  [0.0197, -0.036, 0.0],
  [0.0433, -0.009, 0.0],
  [0.0433, 0.009, 0.0],
  [0.0236, 0.0946, 0.0],
  [-0.0118, 0.045, 0.

In [64]:
ndjson_filepath

'/Users/loicregne/code/rs-uk/raw_data/full_simplified_face.ndjson'

In [65]:
# Where we want to save the output file and a name is given
output_file = path_data + '/' + 'processed_' + file_name
output_file

'/Users/loicregne/code/rs-uk/raw_data/processed_full_simplified_face.ndjson'

In [66]:
# Function to save the dictionary file for the processed images
def save_images_dict_to_json(list_drawings, output_file):
    with open(output_file, 'w') as json_file:
        json.dump(list_drawings, json_file)

In [67]:
# Function to save the drawings in the list to an ndjson file locally
def save_drawings_to_ndjson_local(list_drawings: list, output_file: str) -> None:
    '''
        - list_drawings: contains a dictionary for each drawing
        - output_file: the complete filepath to the target file to save/create (.ndjson)
    '''
    with open(output_file, 'w') as ndjson_file:
        # Write each drawing's dict to the file as a new line
        for dict_drawing in list_drawings:
            ndjson_file.write(str(dict_drawing) + '\n')

In [77]:
%%time
list_drawings = drawings_extraction_v2(ndjson_filepath, 'all')

CPU times: user 8.98 s, sys: 1.04 s, total: 10 s
Wall time: 10.5 s


In [78]:
%%time
save_images_dict_to_json(list_drawings, output_file)

CPU times: user 20.1 s, sys: 498 ms, total: 20.6 s
Wall time: 21.3 s


In [79]:
%%time
save_drawings_to_ndjson_local(list_drawings, output_file)

CPU times: user 4.95 s, sys: 498 ms, total: 5.45 s
Wall time: 5.99 s


In [ ]:
def upload_blob(bucket_name, file_path, file_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    blob.upload_from_filename(file_path + '/' + file_name)

In [ ]:
bucket_name = 'quickdraw-simplified-processed'
file_path = path_data
file_name = 'processed_full_simplified_face.json'

upload_blob(bucket_name, file_path, file_name)